# Nulls, Errors, and Exceptions - The Scala Way

Blah blah blah

### --- Nulls ---

If you have any function, can you tell from it's type signature if it might return a null?  Do you need to write code and tests to check for nulls?  In Scala, we try to avoid nulls completely.  We have an alternative, the __[Option](https://www.scala-lang.org/api/2.12.8/scala/Option.html)__ class (side note - Java 8 borrowed this concept to create it's `Optional` class).  Of the three classes we will be discussing, this one is the simplest.  It has a type signature that takes one parameter and looks like `Option[A]`.  A null is represented by a `None[A]`, which holds zero elements of type A.  Any non-null value is represented by a `Some[A]`, which holds one element of type A, namely the value.

In [8]:
val option1: Option[String] = None
val option2: Option[String] = Some("foobar")

option1: Option[String] = None
option2: Option[String] = Some("foobar")

An option can also be constructed by in the following way, based on a variable that may or may not be null.  This is useful for wrapping functions which are known to possibly return a null value.

In [7]:
val optionVar1: String = null
val optionVar2: String = "foobar"

Option(optionVar1)
Option(optionVar2)

optionVar3: String = null
optionVar4: String = "foobar"
res6_2: Option[String] = None
res6_3: Option[String] = Some("foobar")

blah blah

In [10]:
option1.map(_.size)
option2.map(_.size)

res9_0: Option[Int] = None
res9_1: Option[Int] = Some(6)

In [9]:
def optionFunction1Pass(x: String): Option[Int] = Some(x.size)
def optionFunction2Pass(x: Int): Option[Int] = Some(x * 3)
def optionFunction3Pass(x: Int): Option[Double] = Some(x.toDouble)
def optionFunction4Pass(x: Double): Option[String] = Some(x.toString)

option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Pass)
  .flatMap(optionFunction4Pass)

defined function optionFunction1Pass
defined function optionFunction2Pass
defined function optionFunction3Pass
defined function optionFunction4Pass
res8_4: Option[String] = Some("18.0")

In [11]:
def optionFunction3Fail(x: Int): Option[Double] = None

option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Fail)
  .flatMap(optionFunction4Pass)

defined function optionFunction3Fail
res10_1: Option[String] = None

### --- Errors ---

In Scala, we usually deal with a method that can return a value or some kind of error using the __[Either](https://www.scala-lang.org/api/2.12.8/scala/util/Either.html)__ monad.  It has a type signature that takes two parameters and looks like `Either[A,B]`.  It is implemented by either a `Left[A]` for the unhappy path, or a `Right[B]` for the happy path.

In [1]:
case class Error(message: String)

val either1: Either[Error, Int] = Left(Error("error"))
val either2: Either[Error, Int] = Right(123)

defined class Error
either1: Either[Error, Int] = Left(Error("error"))
either2: Either[Error, Int] = Right(123)

Either is right biased, meaning operators like `map` and `flatMap` only operate on a Right.  For example, the type signature for map is `map[C](B => C): Either[A,C]`.  Only the value within a Right will be operated on.  The value within a Left will be unchanged.  Either way, the type signature of the Either has changed.

In [2]:
either1.map(_.toDouble)
either2.map(_.toDouble)

res1_0: Either[Error, Double] = Left(Error("error"))
res1_1: Either[Error, Double] = Right(123.0)


The compositional beauty comes in when you have a series of functions, each of which can produce an error.  You want to stop on the first error.  Normally this would look like a bunch of nested `if` statements.  With Either, you can call the first function, then flatMap with the second, flatMap with the third, etc.  If any function returns a Left, any flatMaps after will have no impact.  If there are no Lefts, then the final output will be a Right.

`val foo: Either[Error, Int] = function1(123).flatMap(function2).flatMap(function3)`

In [3]:
def eitherFunc1Pass(x: Int): Either[Error, Int] = Right(x + 321)
def eitherFunc2Pass(x: Int): Either[Error, Int] = Right(x * 2)
def eitherFunc3Pass(x: Int): Either[Error, Double] = Right(x.toDouble)
def eitherFunc4Pass(x: Double): Either[Error, String] = Right(x.toString)

either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Pass)
  .flatMap(eitherFunc4Pass)

defined function eitherFunc1Pass
defined function eitherFunc2Pass
defined function eitherFunc3Pass
defined function eitherFunc4Pass
res2_4: Either[Error, String] = Right("888.0")

In [6]:
def eitherFunc3Fail(x: Int): Either[Error, Double] = Left(Error("function 3 failed"))

either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Fail)
  .flatMap(eitherFunc4Pass)

defined function eitherFunc3Fail
res5_1: Either[Error, String] = Left(Error("function 3 failed"))

### --- Exceptions ---

blah blah